<a href="https://colab.research.google.com/github/bikaneal/ml-thesis/blob/main/request.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymorphy3

In [ ]:
!pip install -U assemblyai

In [ ]:
# Start by making sure the `assemblyai` package is installed.
# If not, you can install it by running the following command:
# pip install -U assemblyai
#
# Note: Some macOS users may need to use `pip3` instead of `pip`.

import assemblyai as aai

# Replace with your API key
aai.settings.api_key = "a33634c5dea14943a8826558e6c994ef"

config = aai.TranscriptionConfig(language_code="ru", speaker_labels=True)
# You can also transcribe a local file by passing in a file path
FILE_URL = open('/content/drive/MyDrive/Colab_Notebooks/make/gametest.mp3', 'rb').read()

transcriber = aai.Transcriber()
transcript = transcriber.transcribe(FILE_URL, config=config)

with open('transcript.txt', 'w') as file:
    for utterance in transcript.utterances:
        file.write(f"Speaker {utterance.speaker}: {utterance.text}n")

In [ ]:
import pandas as pd
import numpy as np
from flask import Flask, request, jsonify
import pymorphy3
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import pickle
import os

nltk.download('stopwords')
nltk.download('punkt')

model = Word2Vec.load('/content/drive/MyDrive/Colab_Notebooks/models/word2vec_class/word2vec-classification-50')

with open('/content/drive/MyDrive/Colab_Notebooks/models/word2vec_class/classifier.pkl', 'rb') as file:
    classifier = pickle.load(file)

def preprocess(text, stop_words, punctuation_marks, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return preprocessed_text

morph = pymorphy3.MorphAnalyzer()
stop_words = stopwords.words('russian')
punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...', '«', '»']

def document_vector(word2vec_model, doc):
    # Отфильтровываем слова, которых нет в модели
    doc = [word for word in doc if word in word2vec_model.wv]
    if len(doc) == 0:
        return np.zeros(word2vec_model.vector_size, dtype=np.float32)

    # Считаем среднее всех векторов слов документа
    return np.mean(word2vec_model.wv[doc], axis=0)

def get_vector(model, tokenized_text):
    return document_vector(model, tokenized_text)

def process_text(text):
    speakers = {}
    lines = text.split('n')  # каждая новая строка считается репликой

    for line in lines:
        if not line.strip():
            continue  # Пропускаем пустые строки

        try:
            speaker, utterance = line.split(': ', 1)
        except ValueError:  # если строка не содержит спикера
            continue

        speaker = speaker.strip()
        utterance = utterance.strip()

        if speaker not in speakers:
            speakers[speaker] = []

        # Предобработка реплики перед подачей в модель
        preprocessed_utterance = preprocess(utterance, stop_words, punctuation_marks, morph)

        # Получение вектора для отдельно взятой реплики
        new_vector = get_vector(model, preprocessed_utterance)
        new_vector = np.array([new_vector])

        # Получение предсказания модели
        prediction = classifier.predict(new_vector)

        if prediction:
            # Принадлежность реплики данному спикеру уже гарантирована, добавляем ее в список
            speakers[speaker].append(utterance)

    return speakers

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
data = open('/content/transcript.txt').read()
data
lines = data.split('n')
for line in lines:
      if not line.strip():
          continue  # Пропускаем пустые строки

      try:
          speaker, utterance = line.split(': ', 1)
      except ValueError:  # если строка не содержит спикера
          continue

      speaker = speaker.strip()
      utterance = utterance.strip()


IndentationError: unexpected indent (<ipython-input-27-56b74fc9aede>, line 4)

In [ ]:
process_text(data)

{'Speaker A': ['А вот тоже я отправила сообщение тут же. А, ну вы пишете 13, предлагаете, а что насчёт 1, 2, что вы думаете по этому поводу?',
  'Что, мы тогда ставим 21 или другие варианты есть? Так, ну 2... Юля и Анна пишут 21. Галина? 29? Нет, подождите, у них максимум 20 ставка.',
  'Но если они пойдут во банку, у них 20, а мы 21 поставим Ну.',
  'Что команда 2 всегда ставит 20, 22, такие. Мне кажется, надо меньше 10 тоже, я согласна. Это хорошо работает, когда они жгут ресурсы, и потом мы с ними работаем. Мне кажется, или 2, или... Я, конечно, за 2, чтобы они осторожны. Но вторая в начале у них была 23 ставка.',
  'Мне кажется, они пойдут в лобанк, а нужно столько, чтобы перекрыть их.',
  'Давайте сорок пять тогда, да?',
  'Ну тогда давайте единицу.'],
 'Speaker B': ['Да, нужно точно Галина, вы поставите? Спасибо Супер Да, точно.',
  'Мы... Поздравляю в одном шаге от победы. Так, я предлагаю поставить семь.',
  'Прости, я тебя вообще не слышу.',
  'Да, мы таким же составом. Что, м

In [ ]:
import requests
import json

# URL вашего Flask приложения
url = 'http://ml-thesis-bikanel.amvera.io/analyze'

# Открыть файл в бинарном режиме
with open('transcript.txt', 'rb') as f:
    file_content = f.read()

# Создать запрос с multipart/form-data
response = requests.post(url, files={'file': file_content})

# Обработать ответ сервера
decoded_response = response.json()

# Преобразование decoded_response в строку формата JSON
json_string = json.dumps(decoded_response, ensure_ascii=False, indent=4)

# Запись строки JSON в файл
with open('result.txt', 'w', encoding='utf-8') as file:
    file.write(json_string)

print('Готово. Данные загрузились в файл result.txt')


Готово. Данные загрузились в файл result.txt
